In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxis-experiment")

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe("./data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("./data/green_tripdata_2021-02.parquet")

In [ ]:
df_train

In [ ]:
df_val

In [ ]:
#Feature Engineering
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [ ]:
dv = DictVectorizer()

categorical = ["PU_DO"] #["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)


val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [ ]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

# preprocessing done!

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

- Lasso(alpha=0.001): 6.583
- Ridge(alpha=0.001): 5.996

In [ ]:
# with open("./models/lin_reg.bin", "wb") as f_out:
#     pickle.dump((dv, lr), f_out)

In [ ]:
# mlflow_experiemnt example
with mlflow.start_run():
    mlflow.set_tag("developer", "aiden")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("./models/las_reg.bin", "wb") as f_out:
        pickle.dump((dv, ls), f_out)
    # from video 2.4 - logging model method1
    mlflow.log_artifact(local_path="./models/las_reg.bin", artifact_path="models_pickle")

In [ ]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, y_train)
valid = xgb.DMatrix(X_val, y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse",rmse)

        return {"loss": rmse, "status":STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) - [0.o5, 1]
    'reg_aplha': hp.loguniform('reg_aplha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_wight': hp.loguniform('min_child_wight', -1, 3),
    'objective': 'reg:linear',
    'seed':42,
    }

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [ ]:
params = {
    'learning_rate': 0.20072438443168863,
    'max_depth': 53,
    'min_child_wight': 2.1112596906064938,
    'objective': 'reg:linear',
    'reg_aplha': 0.04888540530278026,
    'reg_lambda': 0.18312272392405518,
    'seed': 42,
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
#from video 2.4 - logiing models method2
with mlflow.start_run():
    params = {
        'learning_rate': 0.20072438443168863,
        'max_depth': 53,
        'min_child_wight': 2.1112596906064938,
        'objective': 'reg:linear',
        'reg_aplha': 0.04888540530278026,
        'reg_lambda': 0.18312272392405518,
        'seed': 42,
    }

    mlflow.log_params(params)

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
# use saved model path1 as python fun in pandas dfs
logged_model = 'runs:/ef251edfc00d490db12c90fe5a05ed31/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
# path2 xgb model
xgb_model = mlflow.xgboost.load_model(logged_model)

In [ ]:
y_pred = xgb_model.predict(valid)

In [ ]:
y_pred[:10]

# video 2.5

In [ ]:
mlflow.xgboost.autolog()

with mlflow.start_run():
    params = {
        'learning_rate': 0.20072438443168863,
        'max_depth': 53,
        'min_child_wight': 2.1112596906064938,
        'objective': 'reg:linear',
        'reg_aplha': 0.04888540530278026,
        'reg_lambda': 0.18312272392405518,
        'seed': 42,
    }

    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor

from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (LinearRegression, GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "green_tripdata_2021-02.parquetv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)